In [1]:
import geocoder


def get_lat_long_by_county(county_name):
    location = geocoder.arcgis(county_name + ", USA")
    if location.ok:
        return (location.latlng[0], location.latlng[1])
    else:
        return None


def get_county_and_state_by_lat_long(latitude, longitude):
    location = geocoder.arcgis([latitude, longitude], method="reverse")
    if location.ok:
        return location.address
    else:
        return None


def round_latlng_str(latlng, precision=4):
    return f"({round(latlng[0], precision)}, {round(latlng[1], precision)})"

In [2]:
import os

data_dir = "./raw_weather_data"
filenames = sorted(os.listdir(data_dir))

county_latlng_map = {}
failed_counties = []

for i, str in enumerate(filenames):
    state, county = str.replace(".pkl.tw", "").split(".")
    address = county.upper() + ", " + state.upper()
    latlng = get_lat_long_by_county(address)
    if latlng is not None:
        county_latlng_map[address] = latlng
        inverse_address = get_county_and_state_by_lat_long(latlng[0], latlng[1])
        rounded_latlng = round_latlng_str(latlng)
        print(
            f"#{i:>4d}/{len(filenames)} | {address:<25s} | {rounded_latlng:<20s} | Inverse Geocoding: {inverse_address}"
        )
    else:
        failed_counties.append(address)
        print(f"#{i:<4d} | {address:<30s} | Failed")

#   0/261 | COFFEE, ALABAMA           | (31.4026, -85.9882)  | Inverse Geocoding: 189 County Road 500, Elba, Alabama, 36323
#   1/261 | COLBERT, ALABAMA          | (32.28, -85.2376)    | Inverse Geocoding: 22-34 Colbert Rd, Hatchechubbee, Alabama, 36858
#   2/261 | CULLMAN, ALABAMA          | (34.1738, -86.843)   | Inverse Geocoding: 101-169 4th St SW, Cullman, Alabama, 35055
#   3/261 | JACKSON, ALABAMA          | (31.5207, -87.8958)  | Inverse Geocoding: 1000-1016 Coffeeville Rd, Jackson, Alabama, 36545
#   4/261 | JEFFERSON, ALABAMA        | (30.5655, -88.2327)  | Inverse Geocoding: 36582, Theodore, Alabama
#   5/261 | LAUDERDALE, ALABAMA       | (34.9014, -87.654)   | Inverse Geocoding: 35634, Florence, Alabama
#   6/261 | MADISON, ALABAMA          | (34.705, -86.751)    | Inverse Geocoding: Blue Bell Dr, Madison, Alabama, 35758
#   7/261 | MARSHALL, ALABAMA         | (32.4343, -87.325)   | Inverse Geocoding: County Road 179, Marion Junction, Alabama, 36759
#   8/261 | MARICOPA, AR

In [7]:
import pandas as pd

county_latlng_map = pd.DataFrame(
    {
        "CountyName": list(county_latlng_map.keys()),
        "Latitude": [latlng[0] for latlng in county_latlng_map.values()],
        "Longitude": [latlng[1] for latlng in county_latlng_map.values()],
    }
)

county_latlng_map

,CountyName,Latitude,Longitude
0,"COFFEE, ALABAMA",31.402630,-85.988210
1,"COLBERT, ALABAMA",32.280036,-85.237594
2,"CULLMAN, ALABAMA",34.173790,-86.843012
3,"JACKSON, ALABAMA",31.520672,-87.895819
4,"JEFFERSON, ALABAMA",30.565535,-88.232736
...,...,...,...
256,"CLARK, WISCONSIN",46.326147,-90.659637
257,"DANE, WISCONSIN",43.250437,-89.497536
258,"MARATHON, WISCONSIN",44.899596,-89.783847
259,"MILWAUKEE, WISCONSIN",43.041072,-87.909421


In [8]:
county_latlng_map.to_csv("county_latlong_map.csv", index=False)